In [115]:
import torch
import pandas as pd

import lightning as L
from torch.utils.data import TensorDataset, DataLoader
import model as m


model = m.LightningLSTM()

In [116]:
df =  pd.read_csv("Gasleak Data Sets/sensor_readings.csv")
df.head()

,Unnamed: 0,time,111111_ 40.595561_-105.14055_3,111111_ 40.596108_-105.140583_4,111111_40.595556_-105.140069_2,111111_40.596114_-105.140075_1,222222_ 40.596108_-105.140583_4,222222_40.595556_-105.140069_2,222222_40.595561_-105.14055_3,222222_40.596114_-105.140075_1,...,444444_40.595881_-105.139686_1,444444_40.595947_-105.139833_4,555555_40.595542_-105.139211_2,555555_40.595547_-105.139714_3,555555_40.596089_-105.139144_1,555555_40.596097_-105.139678_4,666666_40.595542_-105.139211_2,666666_40.595547_-105.139714_3,666666_40.596089_-105.139144_1,666666_40.596097_-105.139678_4
0,1,1681776002,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,...,1120.0,735.0,1088.0,991.5,931.0,1040.0,1040.0,1111.0,1080.0,1089.0
1,2,1681776003,1033.0,1051.0,1163.0,974.0,993.0,1095.0,1044.0,1152.0,...,1104.5,765.0,1090.0,984.0,928.5,1043.0,1052.0,1105.5,1075.0,1095.0
2,3,1681776004,1022.0,1051.0,1160.0,975.5,974.0,1081.0,1049.0,1133.0,...,1089.0,762.0,1095.0,993.0,926.0,1039.0,1033.0,1100.0,1021.0,1093.0
3,4,1681776005,1034.0,1050.0,1165.0,977.0,988.0,1084.0,1019.0,1128.0,...,1082.0,762.0,1100.0,999.0,929.0,1047.5,1014.0,1107.0,1059.5,1110.0
4,5,1681776006,1037.5,1049.0,1155.0,977.0,971.0,1084.0,1027.0,1144.0,...,1070.0,763.0,1100.0,1004.5,944.0,1056.0,1019.0,1098.0,1098.0,1107.5


In [117]:
df.shape


(83413, 26)

In [118]:
leak_locations_and_rateDF = pd.read_csv("Gasleak Data Sets/leak_locations_and_rate.csv")
leak_locations_and_rateDF.head()


,EventID,NumberSourcesLeaking,LeakPointId,Latitude,Longitude,EmissionCategory,UTCStart,UTCEnd,Duration,LeakRate,BFT,tStart,tEnd
0,40,2,4T-31,40.595830,-105.139824,Fugitive,2023-04-18_00:00:35.000000,2023-04-18_00:15:35.000000,900,36.636270,ReferenceMeteredEvent,1681776035,1681776935
1,40,2,5S-26,40.595924,-105.139390,Fugitive,2023-04-18_00:00:35.000000,2023-04-18_00:15:35.000000,900,64.621765,ReferenceMeteredEvent,1681776035,1681776935
2,41,2,4W-27,40.595936,-105.140270,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_01:00:35.000000,2700,55.385376,ReferenceMeteredEvent,1681776935,1681779635
3,41,2,4S-43,40.595642,-105.140350,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_01:00:35.000000,2700,24.345577,ReferenceMeteredEvent,1681776935,1681779635
4,42,2,4T-31,40.595830,-105.139824,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_00:45:35.000000,1800,36.636270,ReferenceMeteredEvent,1681776935,1681778735


In [119]:
weather_df = pd.read_csv("Gasleak Data Sets/weather_data.csv")

In [120]:
total_data  = df.join(weather_df)
total_data = total_data.drop('timestamp', axis=1)
total_data.head()

,Unnamed: 0,time,111111_ 40.595561_-105.14055_3,111111_ 40.596108_-105.140583_4,111111_40.595556_-105.140069_2,111111_40.596114_-105.140075_1,222222_ 40.596108_-105.140583_4,222222_40.595556_-105.140069_2,222222_40.595561_-105.14055_3,222222_40.596114_-105.140075_1,...,555555_40.596097_-105.139678_4,666666_40.595542_-105.139211_2,666666_40.595547_-105.139714_3,666666_40.596089_-105.139144_1,666666_40.596097_-105.139678_4,Barometric_Pressure,Humidity,Temperature,Wind_Direction,Wind_Speed
0,1,1681776002,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,...,1040.0,1040.0,1111.0,1080.0,1089.0,827.064433,32.200298,10.971487,306.335774,1.836502
1,2,1681776003,1033.0,1051.0,1163.0,974.0,993.0,1095.0,1044.0,1152.0,...,1043.0,1052.0,1105.5,1075.0,1095.0,827.073897,32.200297,11.116626,311.523873,1.875021
2,3,1681776004,1022.0,1051.0,1160.0,975.5,974.0,1081.0,1049.0,1133.0,...,1039.0,1033.0,1100.0,1021.0,1093.0,827.301068,32.138772,10.861056,324.694315,1.966547
3,4,1681776005,1034.0,1050.0,1165.0,977.0,988.0,1084.0,1019.0,1128.0,...,1047.5,1014.0,1107.0,1059.5,1110.0,827.073913,32.100913,11.012511,316.473451,2.060548
4,5,1681776006,1037.5,1049.0,1155.0,977.0,971.0,1084.0,1027.0,1144.0,...,1056.0,1019.0,1098.0,1098.0,1107.5,827.092847,31.999946,10.969914,315.187093,1.947818


In [121]:
leak_locations_and_rateDF.head()

,EventID,NumberSourcesLeaking,LeakPointId,Latitude,Longitude,EmissionCategory,UTCStart,UTCEnd,Duration,LeakRate,BFT,tStart,tEnd
0,40,2,4T-31,40.595830,-105.139824,Fugitive,2023-04-18_00:00:35.000000,2023-04-18_00:15:35.000000,900,36.636270,ReferenceMeteredEvent,1681776035,1681776935
1,40,2,5S-26,40.595924,-105.139390,Fugitive,2023-04-18_00:00:35.000000,2023-04-18_00:15:35.000000,900,64.621765,ReferenceMeteredEvent,1681776035,1681776935
2,41,2,4W-27,40.595936,-105.140270,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_01:00:35.000000,2700,55.385376,ReferenceMeteredEvent,1681776935,1681779635
3,41,2,4S-43,40.595642,-105.140350,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_01:00:35.000000,2700,24.345577,ReferenceMeteredEvent,1681776935,1681779635
4,42,2,4T-31,40.595830,-105.139824,Fugitive,2023-04-18_00:15:35.000000,2023-04-18_00:45:35.000000,1800,36.636270,ReferenceMeteredEvent,1681776935,1681778735


In [131]:
leak_locations_and_rateDF.value_counts()
# leak_locations_and_rateDF = leak_locations_and_rateDF[["tStart", "tEnd"]]
# leak_locations_and_rateDF.head()

tStart      tEnd      
1681853039  1681853938    4
1681801899  1681802799    3
1681800099  1681800998    3
1681802799  1681803699    3
1681799198  1681800099    3
1681796498  1681799198    3
1681800998  1681801899    3
1681839339  1681841139    2
1681838440  1681839339    2
1681837540  1681844740    2
1681837539  1681838440    2
1681841139  1681842039    2
1681835740  1681837540    2
            1681836639    2
1681835560  1681835740    2
1681776935  1681778735    2
1681842039  1681844740    2
1681804800  1681835320    2
1681803699  1681804800    2
1681844740  1681851967    2
1681851967  1681852147    2
1681852312  1681852378    2
1681836639  1681837539    2
1681776035  1681776935    2
1681782335  1681785035    2
1681779635  1681782335    2
1681778735  1681779635    2
1681784135  1681785036    2
1681795839  1681796019    2
1681776935  1681779635    2
1681779635  1681784135    2
1681852558  1681853039    1
1681852378  1681852559    1
1681852559  1681852618    1
1681852618  1681852798   

In [123]:
leak_locations_and_rateDF.shape

(98, 2)

In [124]:
# print(len(total_data))
# idx = 0
# for i in range(len(total_data[:5])):
#     t =  total_data.at[i, 'time']
#     if leak_locations_and_rateDF.at[idx,'tStart'] <= t and t <= leak_locations_and_rateDF.at[idx,'tEnd']:
#         total_data.at[i, 'in_range'] = 1
#     elif idx+1 < len(leak_locations_and_rateDF) and leak_locations_and_rateDF.at[idx+1,'tStart'] <= t and t <= leak_locations_and_rateDF.at[idx+1,'tEnd']:
#         idx+=1
#     else:
#         total_data.at[i, 'in_range'] = 0


In [125]:
idx = 0
t = total_data.at[idx,'time']
for index, row in leak_locations_and_rateDF.iterrows():
    while t < row['tStart']:
        total_data.at[idx,'in_range'] = 0
        idx += 1
        t = total_data.at[idx,'time']
    while t <= row['tEnd']:
        total_data.at[idx,'in_range'] = 1
        idx += 1
        t = total_data.at[idx,'time']
for i in range(idx, len(total_data)):
    total_data.at[i,'in_range'] = 0



In [126]:
total_data[:5].head()

,Unnamed: 0,time,111111_ 40.595561_-105.14055_3,111111_ 40.596108_-105.140583_4,111111_40.595556_-105.140069_2,111111_40.596114_-105.140075_1,222222_ 40.596108_-105.140583_4,222222_40.595556_-105.140069_2,222222_40.595561_-105.14055_3,222222_40.596114_-105.140075_1,...,666666_40.595542_-105.139211_2,666666_40.595547_-105.139714_3,666666_40.596089_-105.139144_1,666666_40.596097_-105.139678_4,Barometric_Pressure,Humidity,Temperature,Wind_Direction,Wind_Speed,in_range
0,1,1681776002,1030.0,1049.0,1166.0,964.0,1023.0,1090.5,1039.5,1153.0,...,1040.0,1111.0,1080.0,1089.0,827.064433,32.200298,10.971487,306.335774,1.836502,0.0
1,2,1681776003,1033.0,1051.0,1163.0,974.0,993.0,1095.0,1044.0,1152.0,...,1052.0,1105.5,1075.0,1095.0,827.073897,32.200297,11.116626,311.523873,1.875021,0.0
2,3,1681776004,1022.0,1051.0,1160.0,975.5,974.0,1081.0,1049.0,1133.0,...,1033.0,1100.0,1021.0,1093.0,827.301068,32.138772,10.861056,324.694315,1.966547,0.0
3,4,1681776005,1034.0,1050.0,1165.0,977.0,988.0,1084.0,1019.0,1128.0,...,1014.0,1107.0,1059.5,1110.0,827.073913,32.100913,11.012511,316.473451,2.060548,0.0
4,5,1681776006,1037.5,1049.0,1155.0,977.0,971.0,1084.0,1027.0,1144.0,...,1019.0,1098.0,1098.0,1107.5,827.092847,31.999946,10.969914,315.187093,1.947818,0.0


In [127]:
total_data.tail()

,Unnamed: 0,time,111111_ 40.595561_-105.14055_3,111111_ 40.596108_-105.140583_4,111111_40.595556_-105.140069_2,111111_40.596114_-105.140075_1,222222_ 40.596108_-105.140583_4,222222_40.595556_-105.140069_2,222222_40.595561_-105.14055_3,222222_40.596114_-105.140075_1,...,666666_40.595542_-105.139211_2,666666_40.595547_-105.139714_3,666666_40.596089_-105.139144_1,666666_40.596097_-105.139678_4,Barometric_Pressure,Humidity,Temperature,Wind_Direction,Wind_Speed,in_range
83408,83409,1681862395,853.0,901.0,990.0,813.0,791.0,911.0,887.0,943.0,...,955.0,1020.0,949.0,987.0,822.189666,33.490782,12.381869,323.544266,7.610777,0.0
83409,83410,1681862396,852.5,899.0,1007.0,801.0,788.5,900.0,890.0,931.0,...,954.0,1018.0,955.0,986.0,822.426273,33.528638,12.519112,325.179907,7.073987,0.0
83410,83411,1681862397,852.0,901.0,1011.0,811.0,786.0,909.0,899.0,945.0,...,961.0,1017.0,938.0,982.0,822.199136,33.489204,12.501765,324.447291,7.718912,0.0
83411,83412,1681862398,859.0,912.0,1010.0,811.0,764.0,909.0,892.0,959.0,...,963.0,1026.0,948.0,990.0,822.274835,33.501820,12.485986,324.072430,7.677212,0.0
83412,83413,1681862399,855.0,900.0,1010.0,806.0,786.0,913.0,901.0,962.0,...,953.0,1017.0,846.0,971.0,822.293750,33.503397,12.616926,327.718573,7.833760,0.0


In [128]:
leak_locations_and_rateDF.tail()

,tStart,tEnd
93,1681853000,1681853039
94,1681853039,1681853938
95,1681853039,1681853938
96,1681853039,1681853938
97,1681853039,1681853938


In [129]:
total_data['in_range'].value_counts()

in_range
1.0    75631
0.0     7782
Name: count, dtype: int64

In [130]:
len(total_data)

83413